# Imports

In [24]:
import numpy as np
import nibabel as nib
import dipy.reconst.dti as dti
from dipy.data import fetch_stanford_hardi
from dipy.data import read_stanford_hardi
from dipy.segment.mask import median_otsu
from dipy.reconst.dti import fractional_anisotropy, color_fa

from dipy.data import get_sphere
from dipy.viz import window, actor

ImportError: cannot import name 'window'

# Fetch Data

In [4]:
fetch_stanford_hardi()

Creating new folder C:\Users\boywi\.dipy\stanford_hardi
Download Progress: [########################################] 100.00%  of 87.15 MBDownloading "HARDI150.bval" to C:\Users\boywi\.dipy\stanford_hardi
Download Progress: [########################################] 100.00%  of 0.00 MBDownloading "HARDI150.bvec" to C:\Users\boywi\.dipy\stanford_hardi
Download Progress: [########################################] 100.00%  of 0.00 MBFiles successfully downloaded to C:\Users\boywi\.dipy\stanford_hardi


({'HARDI150.nii.gz': ('https://stacks.stanford.edu/file/druid:yx282xq2090/dwi.nii.gz',
   '0b18513b46132b4d1051ed3364f2acbc'),
  'HARDI150.bval': ('https://stacks.stanford.edu/file/druid:yx282xq2090/dwi.bvals',
   '4e08ee9e2b1d2ec3fddb68c70ae23c36'),
  'HARDI150.bvec': ('https://stacks.stanford.edu/file/druid:yx282xq2090/dwi.bvecs',
   '4c63a586f29afc6a48a5809524a76cb4')},
 'C:\\Users\\boywi\\.dipy\\stanford_hardi')

# Read Data

In [7]:
img, gtab = read_stanford_hardi()

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\boywi\.dipy\stanford_hardi 


# View Data Properties

In [8]:
data = img.get_data()
print('data.shape (%d, %d, %d, %d)' % data.shape)

data.shape (81, 106, 76, 160)


# Mask Image

In [11]:
maskdata, mask = median_otsu(data, vol_idx=range(10, 50), median_radius=3,
                             numpass=1, autocrop=True, dilate=2)
print('maskdata.shape (%d, %d, %d, %d)' % maskdata.shape)

maskdata.shape (71, 87, 62, 160)


# Instantiate TensorModel

In [12]:
tenmodel = dti.TensorModel(gtab)

# Fit Model

In [13]:
tenfit = tenmodel.fit(maskdata)

# Computing anisotropy measures (FA, MD, RGB)
FA is defined as the normalized variance of the eigen-values of the tensor

In [15]:
FA = fractional_anisotropy(tenfit.evals)

# Remove nans

In [16]:
FA[np.isnan(FA)] = 0

# Save FA Images

In [18]:
fa_img = nib.Nifti1Image(FA.astype(np.float32), img.affine)
nib.save(fa_img, 'tensor_fa.nii.gz')

# Save evecs images

In [19]:
evecs_img = nib.Nifti1Image(tenfit.evecs.astype(np.float32), img.affine)
nib.save(evecs_img, 'tensor_evecs.nii.gz')

# Compute the Mea Diffusivity

In [20]:
# method 1
MD1 = dti.mean_diffusivity(tenfit.evals)
nib.save(nib.Nifti1Image(MD1.astype(np.float32), img.affine), 'tensors_md.nii.gz')

# method 2
MD2 = tenfit.md

# Compute colored FA or RGB-map
1. Make sure the FA is scaled between 0 and 1
2. Compute RBG map and save it

In [21]:
FA = np.clip(FA, 0, 1)
RBG = color_fa(FA, tenfit.evecs)
nib.save(nib.Nifti1Image(np.array(255 * RBG, 'uint8'), img.affine), 'tensor_rgb.nii.gz')

# Visualize the tensor ellipsoids of a small rectangular area in an axial slice of the splenium of the corpus callosum

In [22]:

sphere = get_sphere('repulsion724')

# Enables/disables interactive visualization
interactive = False

ren = window.Renderer()

evals = tenfit.evals[13:43, 44:74, 28:29]
evecs = tenfit.evecs[13:43, 44:74, 28:29]

ImportError: cannot import name 'window'